# <font color='Blue'>Análise de Dados com Linguagem Python</font>

## <font color='blue'>Projeto Sienge Arqos</font>


In [2]:
# Imports
import math
import sys, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import watermark 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 

In [3]:
sys.path.append(os.path.abspath(os.path.join('Modulos')))
from estrategia1 import *
from estrategia2 import *
from estrategia3 import *

In [4]:
!pip install -q xlrd

In [5]:
#!pip install -q unidecode
!pip install -q dash-bootstrap-components


## Carregando os Dados


In [7]:
# Importando Arquivo e carregando dados
df = pd.read_excel("dados/Query1.xlsx")


ValueError: Your version of xlrd is 2.0.1. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.

In [ ]:
from unidecode import unidecode

df.columns = [unidecode(col).replace(" ", "_") for col in df.columns]


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# We have null values ? nos temos valores ausentes no data . verdadeiro ou falso para valores ausentes 
df.isna().any()

In [ ]:
# Sum null values " soma valores ausentes' 
df.isna().sum()


### Columns the Drop 

> Iremos remover a coluna ID , Observação gestor , Detalhes para aprovação ,Link do documento,Id 
 > Segundo a dona dos dados tais informações não são relevantes Resp: Liliane Amanda!

In [ ]:
# Iremos remover a coluna ID pois se trata de um identificação pouco importante 
  # Drop deleta coluna "CUSTOMER-ID" axis eixo 1 inplace = a imputar exclucão no date set!  
# Remover as colunas corretamente com os nomes ajustados
df.drop(["Observacao_Gestor", "Detalhes_para_aprovacao:", "Link_dos_documentos_da_negociacao", "ID"], axis=1, inplace=True)


In [ ]:
# Sum null values " soma valores ausentes' 
df.isna().sum()

### Tratando Variavel Numero Sienge 

In [ ]:
print(df.columns)

In [ ]:
df.Numero_SIENGE.isnull().sum()

In [ ]:
df.Numero_SIENGE.isnull().mean()*100

In [ ]:
# Substitui valores NaN por 0 para evitar erro ao converter para int
df["Numero_SIENGE"] = df["Numero_SIENGE"].fillna(0)

# Converte a coluna para inteiro (removendo decimais)
df["Numero_SIENGE"] = df["Numero_SIENGE"].astype(int)

# Converte a coluna para string
df["Numero_SIENGE"] = df["Numero_SIENGE"].astype(str)

# Exibe as primeiras 40 linhas da coluna
#print(df["Numero_SIENGE"].head(40))

In [ ]:
# Porcentagem de valores ausentes na coluna Numero sienge valores foram subistiuidos por 0
df.Numero_SIENGE.isnull().mean()*100

In [ ]:
df.head()

In [ ]:
df['Gestor_Responsavel'].value_counts().head(50)


In [ ]:
df_gestores = df['Gestor_Responsavel'].value_counts().reset_index()
df_gestores.columns = ['Gestor_Responsavel', 'Quantidade']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Criar DataFrame com contagem dos gestores
df_gestores = df['Gestor_Responsavel'].value_counts().reset_index()
df_gestores.columns = ['Gestor_Responsavel', 'Quantidade']

# Gerar uma lista de cores para cada barra
colors = plt.cm.get_cmap("tab20", len(df_gestores))  # Usando uma paleta de 20 cores

# Criar gráfico de barras
plt.figure(figsize=(12, 6))
bars = plt.bar(df_gestores['Gestor_Responsavel'], df_gestores['Quantidade'], color=colors(range(len(df_gestores))))

# Adicionar linha da mediana
plt.axhline(y=mediana, color='red', linestyle='--', label=f'Mediana: {mediana}')


# Ajustar rótulos
plt.xticks(rotation=45, ha='right')
plt.ylabel('Frequência')
plt.title('Quantidade de Ocorrências por Gestor Responsável')

# Adicionar legenda
plt.legend()

# Mostrar gráfico
plt.show()


In [ ]:
df_copia = df.copy()

In [ ]:
# Primeiro, garantir que as colunas de valores estejam no formato numérico
df['Valor_final_negociado'] = pd.to_numeric(df['Valor_final_negociado'], errors='coerce')
df['Valor_final_Inicial_de_Cotacao'] = pd.to_numeric(df['Valor_Inicial_de_Cotacao'], errors='coerce')

# Agrupar pelos gestores e somar os valores
df_agrupado = df.groupby('Gestor_Responsavel')[['Valor_final_Inicial_de_Cotacao', 'Valor_final_negociado']].sum().reset_index()

# Se necessário, reformatar as colunas após o agrupamento
df_agrupado['Valor_final_negociado_formatado'] = df_agrupado['Valor_final_negociado'].apply(lambda x: f'{x:,.2f}')
df_agrupado['Valor_final_Inicial_de_Cotacao_formatado'] = df_agrupado['Valor_final_Inicial_de_Cotacao'].apply(lambda x: f'{x:,.2f}')

# Remover as colunas originais sem formatação
df_agrupado.drop(columns=['Valor_final_negociado', 'Valor_final_Inicial_de_Cotacao'], inplace=True)



### Gerando Grafico de Linhas Para analizar!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ordenar os dados para o gráfico (você pode escolher a ordem desejada)
df_agrupado_sorted = df_agrupado.sort_values(by='Valor_final_negociado_formatado')

# Definir largura das barras
bar_width = 0.35

# Definir a posição das barras
index = np.arange(len(df_agrupado_sorted))

# Criar o gráfico de barras agrupadas
plt.figure(figsize=(12, 6))
plt.bar(index, df_agrupado_sorted['Valor_final_negociado_formatado'], bar_width, label='Valor Final Negociado', color='b')
plt.bar(index + bar_width, df_agrupado_sorted['Valor_final_Inicial_de_Cotacao_formatado'], bar_width, label='Valor Final Cotação Inicial', color='g')

# Adicionar título e rótulos
plt.title('Comparação entre Valor Final Negociado e Valor Final Cotação Inicial por Gestor')
plt.xlabel('Gestor Responsável')
plt.ylabel('Valor (R$)')
plt.xticks(index + bar_width / 2, df_agrupado_sorted['Gestor_Responsavel'], rotation=90)
plt.legend()

# Exibir o gráfico
plt.tight_layout()
plt.show()


In [ ]:
print(df_agrupado_sorted[['Proficiencia', 'Diferença']].isna().sum())


In [ ]:

#df_agrupado.to_csv('E:/ProgramacaoPhyton/DataScience02\DashSiengeAqs/dados/df_agrupado.csv', index=False)
